## EVCharging Data Analysis

This notebook plots the EVCharging session data used in the `EVChargingGym`, including both real historical data as well as synthetic data generated from Gaussian Mixture Models (GMMs) fitted to the real historical data.

As detailed in train_gmm_model.py, the GMMs are fitted to 4 feature dimensions. The 4 features are, in order,
- `'arrival_time'`: minute of day, normalized to [0, 1)
- `'departure_time'`: minute of day, normalized to [0, 1)
- `'estimated_departure_time'`: minute of day, normalized to [0, 1)
- `'requested_energy'`: energy requested; multiply by 100 to get kWh 

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%cd ../..

In [ ]:
from __future__ import annotations

import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.mixture import GaussianMixture

from sustaingym.envs.evcharging import (
    GMMsTraceGenerator, RealTraceGenerator, utils)
from sustaingym.envs.evcharging.train_gmm_model import preprocess


sns.set_style('darkgrid')
SEASONS = ['Summer 2019', 'Fall 2019', 'Spring 2020', 'Summer 2021']

### Plot GMM contours

Creates plots of GMM log-likelihood contours, over-laid on real data scatterplot.

In [ ]:
def plot_gmm_fit(site: utils.SiteStr, n_components: int = 30) -> None:
    """Plot actual arrival/departures against GMM PDF contours.

    We plot the log-likelihood of conditional means of the GMM on a mesh to
    generate contour lines.

    Args:
        site: either 'caltech' or 'jpl'
        n_components: number of components of GMM model,
            default trained GMMs provided with SustainGym have 30 components
    """
    fig, axs = plt.subplots(1, 4, sharey=True, figsize=(12, 3.5), tight_layout=True)

    # Display predicted scores by the model as a contour plot
    arr, dep = np.linspace(0, 1, num=50), np.linspace(0, 1, num=50)
    X, Y = np.meshgrid(arr, dep)  # each shape [50, 50]
    Xr, Yr = X.ravel(), Y.ravel()  # each shape [N], N = 2500
    x_b = np.vstack([Xr, Yr])  # shape [2, N]

    for c, season in enumerate(SEASONS):
        start_date = utils.to_la_dt(utils.DEFAULT_PERIOD_TO_RANGE[season][0])
        end_date = utils.to_la_dt(utils.DEFAULT_PERIOD_TO_RANGE[season][1])

        gmm: GaussianMixture = utils.load_gmm_model(
            site, start_date, end_date, n_components)['gmm']
        assert len(gmm.weights_) == n_components

        # Calculate conditional mean for a=(estimated departure, requested energy)
        # given b=(arrival time, departure time).
        # See equation 2.75 in PRML (Bishop, 2006)
        conditional_means = np.zeros([n_components, 2500, 2])
        for i in range(n_components):
            mu_a = gmm.means_[i, 2:][:, np.newaxis]  # shape [2, 1]
            mu_b = gmm.means_[i, :2][:, np.newaxis]  # shape [2, 1]
            precision_aa = gmm.precisions_[i, 2:, 2:]
            precision_ab = gmm.precisions_[i, 2:, :2]
            cond_a_b = mu_a - np.linalg.pinv(precision_aa) @ precision_ab @ (x_b - mu_b)
            conditional_means[i] = cond_a_b.T

        weights = gmm.weights_[:, np.newaxis, np.newaxis]  # shape [n_components, 1, 1]
        conditional_mean = np.sum(weights * conditional_means, axis=0)  # shape [N, 2]

        gmm_input = pd.DataFrame({
            'arrival_time': Xr,
            'departure_time': Yr,
            'estimated_departure_time': conditional_mean[:, 0],
            'requested_energy (kWh)': conditional_mean[:, 1]
        })
        Z = gmm.score_samples(gmm_input).reshape(X.shape)
        Z[X > Y] = np.nan

        x = X * 288 # rescale to 5-min periods
        y = Y * 288

        # same with arrivals and departures
        df = preprocess(utils.get_real_events(start_date, end_date, site))
        arr_time = df.arrival_time * 288
        dep_time = df.departure_time * 288

        ax = axs[c]
        cp = ax.contourf(x, y, Z, cmap='Oranges', alpha=0.5)
        ax.scatter(arr_time, dep_time, marker='+', alpha=0.2, color='tab:blue')
        ax.set(xlabel='arrival time', title=f'{SEASONS[c]}')
        ax.set_aspect('equal', 'box')
        ax.set_xticks([0, 96, 192, 288], labels=['0:00', '8:00', '16:00', '24:00'])

    axs[0].set_ylabel('departure time')
    axs[0].set_yticks([0, 96, 192, 288], labels=['0:00', '8:00', '16:00', '24:00'])

    cbar_ax = fig.add_axes([1, 0.18, 0.02, 0.66])
    fig.colorbar(cp, cax=cbar_ax)
    cbar_ax.set_ylabel('log-likelihood', rotation=270, labelpad=15)

    os.makedirs('plots/evcharging', exist_ok=True)
    fig.savefig(f'plots/evcharging/gmms_fit_{site}.png', dpi=300, pad_inches=0, bbox_inches='tight')

In [ ]:
plot_gmm_fit('caltech')
plot_gmm_fit('jpl')

### Plot GMM cross log-likelihoods

Calculate the log-likelihoods of the real data under the various GMM models. The significantly lower log-likelihoods on the off-diagonal entries is an indicator of distribution shift.

In [ ]:
def plot_cross_scores() -> None:
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), tight_layout=True)
    cbar_ax = fig.add_axes([1, 0.18, 0.02, 0.66])

    fig.suptitle('30-component GMM Average Log-Likelihood Scores')

    two_word_seasons = [s.replace(' ', '\n') for s in SEASONS]

    for site_idx, site in enumerate(['caltech', 'jpl']):
        dfs, gmms = [], []
        for season in SEASONS:
            start_date = utils.to_la_dt(utils.DEFAULT_PERIOD_TO_RANGE[season][0])
            end_date = utils.to_la_dt(utils.DEFAULT_PERIOD_TO_RANGE[season][1])

            df = preprocess(utils.get_real_events(start_date, end_date, site))
            dfs.append(df)

            gmm = utils.load_gmm_model(site, start_date, end_date, 30)['gmm']
            gmms.append(gmm)

        train_cols = {season: [] for season in two_word_seasons}
        for i, train_season in enumerate(two_word_seasons):
            for j, _ in enumerate(two_word_seasons):
                train_cols[train_season].append(gmms[i].score(dfs[j]))
        cross_scores = pd.DataFrame(train_cols, index=two_word_seasons)

        ax = axs[site_idx]
        sns.heatmap(cross_scores, cmap='Blues', annot=True, fmt='.3g', ax=ax,
                    cbar=(site=='jpl'), vmin=-5, vmax=6,
                    cbar_ax=None if site=='caltech' else cbar_ax)
        ax.set(title=site, xlabel='Training period', ylabel='Testing period')
        ax.tick_params(axis='x', labelrotation=0)
        ax.tick_params(axis='y', labelrotation=0)

    cbar_ax.set_ylabel('log-likelihood', rotation=270, labelpad=15)
    os.makedirs('plots/evcharging', exist_ok=True)
    fig.savefig('plots/evcharging/gmm_log_likelihoods.png', dpi=300, bbox_inches='tight')

In [ ]:
plot_cross_scores()

### Test trace generators

Plot 1 day of events using both the `RealTraceGenerator` and the `GMMsTraceGenerator`.

In [ ]:
def plot_oneday_trace(gen: RealTraceGenerator | GMMsTraceGenerator,
                      filename: str, include_moer: bool = True,
                      seed: int = 123) -> None:
    gen.set_seed(seed)

    df = gen._create_events()
    df['duration'] = df['departure'] - df['arrival']
    df['estimated_duration'] = df['estimated_departure'] - df['arrival']
    df.sort_values(by='station_id', inplace=True)

    print('number of plug-in events:', len(df))
    print('total requested energy (kWh):', df['requested_energy (kWh)'].sum())

    if include_moer:
        fig, axs = plt.subplots(2, 1, sharex=True, figsize=(8, 6), tight_layout=True,
                                gridspec_kw={'height_ratios': [4, 1]})
        ax = axs[0]
    else:
        fig, ax = plt.subplots(1, 1, sharex=True, figsize=(8, 6), tight_layout=True)

    bars = ax.barh(y=df['station_id'], width=df['duration'], left=df['arrival'], alpha=0.7, label='actual')
    ax.barh(y=df['station_id'], width=df['estimated_duration'], left=df['arrival'], alpha=0.4, label='estimated')
    ax.bar_label(bars, label_type='center',
                 labels=df['requested_energy (kWh)'].map(lambda x: f'{x:.2f} kWh'))
    ax.legend()
    ax.set(ylabel='charging station', title='Events')

    if include_moer:
        moer = gen.get_moer()
        moer_actual = moer[:, 0]
        ax = axs[1]
        ax.plot(moer_actual)
        ax.set(ylabel='kg CO$_2$ / kWh', title='MOER')

    ax.set_xticks(ticks=range(48, 288, 48), labels=['4am', '8am', '12pm', '4pm', '8pm'])
    ax.set(xlabel='time')

    os.makedirs('examples/plots/evcharging', exist_ok=True)
    fig.savefig(f'examples/plots/evcharging/{filename}.png', dpi=300, bbox_inches='tight')

In [ ]:
rtg = RealTraceGenerator('caltech', date_period='Summer 2019')
plot_oneday_trace(rtg, 'traces_real_seed124', include_moer=True, seed=124)

In [ ]:
atg = GMMsTraceGenerator('caltech', date_period='Summer 2019', n_components=30)
plot_oneday_trace(atg, 'traces_gmm_seed101', include_moer=False, seed=101)